In [1]:
# Import libraries
import numpy as np
import pandas as pd
from obspy import read
from datetime import datetime
import matplotlib.pyplot as plt
import os

In [2]:
# Create a directory for output files
output_directory = 'NASA Space App'
os.makedirs(output_directory, exist_ok=True)  # Create the directory if it doesn't exist

In [3]:
# Function to apply FFT and plot the results
def apply_fft_and_plot(data, time_data, filename):
    # Apply a window function to the data (Hanning window)
    window = np.hanning(len(data))
    windowed_data = data * window

    # Apply FFT
    fft_data = np.fft.fft(windowed_data)
    fft_freq = np.fft.fftfreq(len(windowed_data), d=np.mean(np.diff(time_data)))

    # Only take the positive frequencies
    pos_mask = fft_freq >= 0
    fft_data = np.abs(fft_data[pos_mask]) / len(data)  # Normalize by length
    fft_freq = fft_freq[pos_mask]

    # Plotting the FFT result
    plt.figure(figsize=(10, 5))
    plt.plot(fft_freq, fft_data, color='blue')
    plt.title(f'FFT of {filename}', fontweight='bold')
    plt.xlabel('Frequency (Hz)')
    plt.ylabel('Magnitude (Normalized)')
    plt.grid()
    plt.xlim(0, np.max(fft_freq))
    plt.ylim(0, np.max(fft_data) * 1.1)
    plt.savefig(os.path.join(output_directory, f'fft_{filename}.png'), bbox_inches='tight')  # Save to file
    plt.close()  # Close the plot to free memory

In [4]:
# Load the seismic event catalog
cat_directory = os.path.join('.', 'data', 'lunar', 'training', 'catalogs')
cat_file = os.path.join(cat_directory, 'apollo12_catalog_GradeA_final.csv')

# Check if the catalog file exists
if not os.path.exists(cat_file):
    print(f"Catalog file does not exist: {cat_file}")
else:
    cat = pd.read_csv(cat_file)

    # Use only the first 5 seismic events for quick analysis
    sample_cat = cat.head(5)

    for index, row in sample_cat.iterrows():
        arrival_time = datetime.strptime(row['time_abs(%Y-%m-%dT%H:%M:%S.%f)'], '%Y-%m-%dT%H:%M:%S.%f')
        arrival_time_rel = row['time_rel(sec)']
        test_filename = row.filename

        # Load the corresponding data
        data_directory = os.path.join('.', 'data', 'lunar', 'training', 'data', 'S12_GradeA')
        csv_file = os.path.join(data_directory, f'{test_filename}.csv')

        # Check if the data file exists
        if not os.path.exists(csv_file):
            print(f"Data file does not exist: {csv_file}")
            continue  # Skip to the next iteration if the file doesn't exist

        data_cat = pd.read_csv(csv_file)

        # Read in time steps and velocities
        csv_times = np.array(data_cat['time_rel(sec)'].tolist())
        csv_data = np.array(data_cat['velocity(m/s)'].tolist())

        # Plot the seismic data
        plt.figure(figsize=(10, 3))
        plt.plot(csv_times, csv_data, color='lightblue', label='Velocity Data')
        plt.axvline(x=arrival_time_rel, c='red', label='', linestyle='--')
        plt.xlim([min(csv_times), max(csv_times)])
        plt.ylabel('Velocity (m/s)')
        plt.xlabel('Time (s)')
        plt.title("Safe Wave's Result", fontweight='bold')
        plt.legend()
        plt.grid(False)  # Disable the grid for a simple background
        plt.savefig(os.path.join(output_directory, f'seismic_{test_filename}.png'), bbox_inches='tight')  # Save to file
        plt.close()  # Close the plot to free memory

        # Apply FFT and plot the result
        apply_fft_and_plot(csv_data, csv_times, test_filename)
